# 멕시코풍 프랜차이즈 chipotle 주문 데이터 분석

[데이터 셋 다운로드](https://drive.google.com/drive/folders/1vgh_YlGJhttIR-4_39VvBq5YEtJFDSBq?usp=sharing)

## 데이터 불러오기

pandas 라이브러리를 활용해 데이터를 읽어 오자.

```pd.read_csv(file_path, sep=, encoding=)```이런 형태 2번 3번 변수는 생략 가능함

<p>

>sep의 경우 구분자를 의미
>encoding은 글자 깨짐 방지 같은 역할임 한글의 경우 cp949를 대부분 사용한다고 함

In [1]:
import pandas as pd

file_path = "data/chipotle.tsv"

df_chipotle = pd.read_csv(file_path, sep='\t') # tap을 기준으로 나눠서 데이터 가져옴
print(df_chipotle.shape) # 행과 열의 수 반환
print(df_chipotle.info()) # 행의 구성, 열의 구성 정보를 나타냄(데이터 타입 같은 것.)
# 보면 order_id, quantity는 int64 , 나머지 3가지는 object타입임.(거의 문자열 의미)
# choice_description 항목은 총 데이터 4622개 중 3376의 데이터가 존재한다는 것이고 1246개의 결측 데이터가 존재 한다는 의미

(4622, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            4622 non-null   int64 
 1   quantity            4622 non-null   int64 
 2   item_name           4622 non-null   object
 3   choice_description  3376 non-null   object
 4   item_price          4622 non-null   object
dtypes: int64(2), object(3)
memory usage: 180.7+ KB
None


In [2]:
df_chipotle.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [3]:
print(df_chipotle.columns)
print("==============================================")
print(df_chipotle.index)

Index(['order_id', 'quantity', 'item_name', 'choice_description',
       'item_price'],
      dtype='object')
RangeIndex(start=0, stop=4622, step=1)


이제 데이터 셋의 피처의 의미는 파악 되었다.

- order_id : 주문 번호
- quantity : 아이템의 주문 수량
- item_name : 주문한 아이템의 이름
- choice_description : 주문한 아이템의 상세 선택 옵션
- item_price : 주문 아이템의 가격 정보


## 피처의 수치적 특징

quantity, item_price의 수치적 특징을 살펴보면 키, 몸무게 같은 어떤 구간의 모든 값을 데이터로 가질 수 있는 **연속형 피처**이다.

피처 형태의 종류

**수치형 피처**

명칭|특징|예시
-----|----|----|
연속형 피처|어떤 구간 안의 모든 값을 데이터로 가질 수 있다.|키 몸무게
비연속형 피처|셀 수 있으며, 일정 구간 안에서 정해진 몇 개의 값을 가져야 한다|나이


**범주형 피처**

명칭|특징|예시
-----|----|----|
순서가 있는 범주형 피처|순서가 있으나 수치는 아님.|학점(A,B,C,D...)
순서가 없는 범주형 피처|데이터가 구분되면서도 순서가 없음|혈액형

### 기초 통계량 파악

```dscribe()```를 통해서 기초 통계량 파악가능 (단, 수치형 피처 And 정수형)

In [4]:
df_chipotle['order_id'] = df_chipotle['order_id'].astype(str)
print(df_chipotle.describe())

          quantity
count  4622.000000
mean      1.075725
std       0.410186
min       1.000000
25%       1.000000
50%       1.000000
75%       1.000000
max      15.000000


위에 통계를 보고 알 수 있는 점은?

mean(평균)값이 1.075임 이걸 통해서 알 수 있는건?
~~한 사람이 같은 메뉴를 여러개 구매하는 경우는 많지 않다(?)~~

In [5]:
print(len(df_chipotle['order_id'].unique())) # order_id의 개수를 출력
print(len(df_chipotle['item_name'].unique()))

1834
50


## 탐색과 시각화

가장 많이 주문한 메뉴 Top 10을 분석해보자

```value_counts()```는 유일한 값별 개수를 반환함 기본 적으로 내림차순으로 반환(오름차순은 ascending=True)

```iteritmems()```는 딕셔너리로 부를 수 있는 함수임 키 값과 벨류 값의 리스트를 가져옴

```enumerate()```는 인덱스 번호, 값을 쌍으로 출력함

In [6]:
def value_count(df, colname, stop):
    '''
    가장 많이 주문한 아이템 TopN 출력하는 메서드
    :param df: 데이터 프레임
    :param colname: 컬럼명(피처)
    :param stop: n 번째
    :return: TopN 항목 출력
    '''

    item_count = df[colname].value_counts()[:stop] # 데이터 프레임의 colname 각 항목 갯수 stop까지 가져옴
    for idx, (val, cnt) in enumerate(item_count.iteritems(), 1):
        print(f"Top{idx} : {val} {cnt}")

In [7]:
value_count(df_chipotle, 'item_name', 10)

Top1 : Chicken Bowl 726
Top2 : Chicken Burrito 553
Top3 : Chips and Guacamole 479
Top4 : Steak Burrito 368
Top5 : Canned Soft Drink 301
Top6 : Steak Bowl 211
Top7 : Chips 211
Top8 : Bottled Water 162
Top9 : Chicken Soft Tacos 115
Top10 : Chips and Fresh Tomato Salsa 110


## 아이템별 주문 개수와 총량

```groupby()```는 데이터 프레임에서 특정 피처를 기준으로 그룹을 생성하며 이를 통해 그룹별 연산을 적용할 수 있음

In [18]:
def item_group_count(df, groupname, colname, option, stop):
    '''
    아이템 그룹별 통계 구하기
    :param df: 데이터 프레임
    :param groupname: 그룹 기준
    :param colname: 컬럼명
    :param stop: N번째
    '''
    if option == 'count':
        item_group_counts = df.groupby(groupname)[colname].count()
        print(item_group_counts[:stop])
    elif option == "sum":
        item_group_counts = df.groupby(groupname)[colname].sum()
        print(item_group_counts[:stop])

In [19]:
item_group_count(df_chipotle, 'item_name', 'order_id', "count", 10)

item_name
6 Pack Soft Drink         54
Barbacoa Bowl             66
Barbacoa Burrito          91
Barbacoa Crispy Tacos     11
Barbacoa Salad Bowl       10
Barbacoa Soft Tacos       25
Bottled Water            162
Bowl                       2
Burrito                    6
Canned Soda              104
Name: order_id, dtype: int64


In [20]:
item_group_count(df_chipotle, 'item_name', 'quantity', "sum", 10)

item_name
6 Pack Soft Drink         55
Barbacoa Bowl             66
Barbacoa Burrito          91
Barbacoa Crispy Tacos     12
Barbacoa Salad Bowl       10
Barbacoa Soft Tacos       25
Bottled Water            211
Bowl                       4
Burrito                    6
Canned Soda              126
Name: quantity, dtype: int64
